<a href="https://colab.research.google.com/github/dinuka-rp/nerdy-panda/blob/main/Prototype/RecSys/Traits_Content_based_NFT_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Content-based Movie Recommender System

https://towardsdatascience.com/how-to-build-from-scratch-a-content-based-movie-recommender-with-natural-language-processing-25ad400eb243

Check this later (not that different): https://medium.com/dataseries/recommendation-system-in-python-4d4cf6d33166 , https://gist.github.com/deansublett/06f010a886831f9dac5b1f9bce4f4229?source=post_page---------------------------

In [4]:
# !pip install rake-nltk

In [3]:
import pandas as pd
# from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
# import nltk

# nltk.download('stopwords')
# from nltk.corpus import stopwords
# nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Get Dataset

In [32]:
from google.colab import drive
drive.mount('drive')

assets_file_path = "/content/drive/MyDrive/Datasets/FYP/bayc-nft-assets"
df = pd.read_csv(assets_file_path, sep='\t')

# df = df[['nft_id','traits_string','asset_contract_address', 'total_rarity']]
df.head()

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


,nft_id,name,asset_description,asset_contract_address,collection_name,collection_description,collection_slug,collection_verification,creator_username,creator_address,creator_profile_img_url,owner_username,owner_address,owner_profile_img_url,traits_string,total_rarity,num_sales,last_sale_timestamp,last_sale_total_price,last_sale_from_account_address,top_bid,open_sea_link,image_url
0,9974,NaN,NaN,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,Bored Ape Yacht Club,The Bored Ape Yacht Club is a collection of 10...,boredapeyachtclub,True,BoredApeYachtClub,0xaba7161a7fb69c88e16ed9f455ce62b791ee4d03,https://storage.googleapis.com/opensea-static/...,mo31985,0x994cf01f34c51426bcb12bd30ff7079e280e1140,https://storage.googleapis.com/opensea-static/...,eyeseyepatch;hatseaman'shat;clothesrainbowsusp...,204.051791,2,2021-05-09T16:46:27,500000000000000000,0x994cf01f34c51426bcb12bd30ff7079e280e1140,NaN,https://opensea.io/assets/0xbc4ca0eda7647a8ab7...,https://lh3.googleusercontent.com/G60LFd7Xhb4-...
1,9973,NaN,NaN,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,Bored Ape Yacht Club,The Bored Ape Yacht Club is a collection of 10...,boredapeyachtclub,True,BoredApeYachtClub,0xaba7161a7fb69c88e16ed9f455ce62b791ee4d03,https://storage.googleapis.com/opensea-static/...,accubed,0x6e0c6bf243232bcfef85a88eb61062626ffc1e2a,https://storage.googleapis.com/opensea-static/...,furdmt;hatgirl'shairpink;eyesrobot;backgroundy...,182.516792,1,2021-05-01T16:24:22,580000000000000000,0x078ad2aa3b4527e4996d087906b2a3da51bba122,NaN,https://opensea.io/assets/0xbc4ca0eda7647a8ab7...,https://lh3.googleusercontent.com/-4Pz6B1FkH8k...
2,9949,NaN,NaN,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,Bored Ape Yacht Club,The Bored Ape Yacht Club is a collection of 10...,boredapeyachtclub,True,BoredApeYachtClub,0xaba7161a7fb69c88e16ed9f455ce62b791ee4d03,https://storage.googleapis.com/opensea-static/...,NaN,0xd3b38f645ac56e0b8ab7b6071a1cfc12a6e8c05d,https://storage.googleapis.com/opensea-static/...,mouthgringoldgrill;furtan;hatseaman'shat;eyes3...,221.356456,7,2021-11-21T10:04:02,85800000000000000000,0x91e1543bf18cc3c7a25e682f9c20cf8bd6f28548,NaN,https://opensea.io/assets/0xbc4ca0eda7647a8ab7...,https://lh3.googleusercontent.com/RcXAgekWgaH5...
3,9972,NaN,NaN,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,Bored Ape Yacht Club,The Bored Ape Yacht Club is a collection of 10...,boredapeyachtclub,True,BoredApeYachtClub,0xaba7161a7fb69c88e16ed9f455ce62b791ee4d03,https://storage.googleapis.com/opensea-static/...,ARC001,0x0b31cc882a2b3937257ccf785b04b7c8f931207b,https://storage.googleapis.com/opensea-static/...,eyeshypnotized;mouthrage;backgroundaquamarine;...,149.528746,1,2021-05-01T11:03:09,150000000000000000,0x02eb75be1e72e988de64f0088d654d8ea1081e87,NaN,https://opensea.io/assets/0xbc4ca0eda7647a8ab7...,https://lh3.googleusercontent.com/vcL_TRKR8FRq...
4,9948,NaN,NaN,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,Bored Ape Yacht Club,The Bored Ape Yacht Club is a collection of 10...,boredapeyachtclub,True,BoredApeYachtClub,0xaba7161a7fb69c88e16ed9f455ce62b791ee4d03,https://storage.googleapis.com/opensea-static/...,MAJE,0xecba5f51925e6ccec26da38dcd7d5305f6bdfbcb,https://storage.googleapis.com/opensea-static/...,hattrippycaptain'shat;furcream;eyesbloodshot;m...,268.580759,2,2021-05-03T02:45:55,1750000000000000000,0xecba5f51925e6ccec26da38dcd7d5305f6bdfbcb,NaN,https://opensea.io/assets/0xbc4ca0eda7647a8ab7...,https://lh3.googleusercontent.com/jMsgIXhlR0Xe...


## Data Cleaning

In [33]:
for index, row in df.iterrows():
    df.at[index,'reference_id'] = row["asset_contract_address"] + "-" + str(row["nft_id"])

df['traits_string'] = df['traits_string'].str.replace(';',' ')

df.head()

,nft_id,name,asset_description,asset_contract_address,collection_name,collection_description,collection_slug,collection_verification,creator_username,creator_address,creator_profile_img_url,owner_username,owner_address,owner_profile_img_url,traits_string,total_rarity,num_sales,last_sale_timestamp,last_sale_total_price,last_sale_from_account_address,top_bid,open_sea_link,image_url,reference_id
0,9974,NaN,NaN,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,Bored Ape Yacht Club,The Bored Ape Yacht Club is a collection of 10...,boredapeyachtclub,True,BoredApeYachtClub,0xaba7161a7fb69c88e16ed9f455ce62b791ee4d03,https://storage.googleapis.com/opensea-static/...,mo31985,0x994cf01f34c51426bcb12bd30ff7079e280e1140,https://storage.googleapis.com/opensea-static/...,eyeseyepatch hatseaman'shat clothesrainbowsusp...,204.051791,2,2021-05-09T16:46:27,500000000000000000,0x994cf01f34c51426bcb12bd30ff7079e280e1140,NaN,https://opensea.io/assets/0xbc4ca0eda7647a8ab7...,https://lh3.googleusercontent.com/G60LFd7Xhb4-...,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d-9974
1,9973,NaN,NaN,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,Bored Ape Yacht Club,The Bored Ape Yacht Club is a collection of 10...,boredapeyachtclub,True,BoredApeYachtClub,0xaba7161a7fb69c88e16ed9f455ce62b791ee4d03,https://storage.googleapis.com/opensea-static/...,accubed,0x6e0c6bf243232bcfef85a88eb61062626ffc1e2a,https://storage.googleapis.com/opensea-static/...,furdmt hatgirl'shairpink eyesrobot backgroundy...,182.516792,1,2021-05-01T16:24:22,580000000000000000,0x078ad2aa3b4527e4996d087906b2a3da51bba122,NaN,https://opensea.io/assets/0xbc4ca0eda7647a8ab7...,https://lh3.googleusercontent.com/-4Pz6B1FkH8k...,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d-9973
2,9949,NaN,NaN,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,Bored Ape Yacht Club,The Bored Ape Yacht Club is a collection of 10...,boredapeyachtclub,True,BoredApeYachtClub,0xaba7161a7fb69c88e16ed9f455ce62b791ee4d03,https://storage.googleapis.com/opensea-static/...,NaN,0xd3b38f645ac56e0b8ab7b6071a1cfc12a6e8c05d,https://storage.googleapis.com/opensea-static/...,mouthgringoldgrill furtan hatseaman'shat eyes3...,221.356456,7,2021-11-21T10:04:02,85800000000000000000,0x91e1543bf18cc3c7a25e682f9c20cf8bd6f28548,NaN,https://opensea.io/assets/0xbc4ca0eda7647a8ab7...,https://lh3.googleusercontent.com/RcXAgekWgaH5...,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d-9949
3,9972,NaN,NaN,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,Bored Ape Yacht Club,The Bored Ape Yacht Club is a collection of 10...,boredapeyachtclub,True,BoredApeYachtClub,0xaba7161a7fb69c88e16ed9f455ce62b791ee4d03,https://storage.googleapis.com/opensea-static/...,ARC001,0x0b31cc882a2b3937257ccf785b04b7c8f931207b,https://storage.googleapis.com/opensea-static/...,eyeshypnotized mouthrage backgroundaquamarine ...,149.528746,1,2021-05-01T11:03:09,150000000000000000,0x02eb75be1e72e988de64f0088d654d8ea1081e87,NaN,https://opensea.io/assets/0xbc4ca0eda7647a8ab7...,https://lh3.googleusercontent.com/vcL_TRKR8FRq...,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d-9972
4,9948,NaN,NaN,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,Bored Ape Yacht Club,The Bored Ape Yacht Club is a collection of 10...,boredapeyachtclub,True,BoredApeYachtClub,0xaba7161a7fb69c88e16ed9f455ce62b791ee4d03,https://storage.googleapis.com/opensea-static/...,MAJE,0xecba5f51925e6ccec26da38dcd7d5305f6bdfbcb,https://storage.googleapis.com/opensea-static/...,hattrippycaptain'shat furcream eyesbloodshot m...,268.580759,2,2021-05-03T02:45:55,1750000000000000000,0xecba5f51925e6ccec26da38dcd7d5305f6bdfbcb,NaN,https://opensea.io/assets/0xbc4ca0eda7647a8ab7...,https://lh3.googleusercontent.com/jMsgIXhlR0Xe...,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d-9948


In [34]:
df.iloc[0]

nft_id                                                                         9974
name                                                                            NaN
asset_description                                                               NaN
asset_contract_address                   0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d
collection_name                                                Bored Ape Yacht Club
collection_description            The Bored Ape Yacht Club is a collection of 10...
collection_slug                                                   boredapeyachtclub
collection_verification                                                        True
creator_username                                                  BoredApeYachtClub
creator_address                          0xaba7161a7fb69c88e16ed9f455ce62b791ee4d03
creator_profile_img_url           https://storage.googleapis.com/opensea-static/...
owner_username                                                              

In [ ]:
# Rake Vectorizer
# # initializing the new column
# df['Key_words'] = ""

# for index, row in df.iterrows():
#     plot = row['Plot']
    
#     # instantiating Rake, by default it uses english stopwords from NLTK
#     # and discards all puntuation characters as well
#     r = Rake()

#     # extracting the words by passing the text
#     r.extract_keywords_from_text(plot)

#     # getting the dictionary with key words as keys and their scores as values
#     key_words_dict_scores = r.get_word_degrees()
    
#     # assigning the key words to the new column for the corresponding movie
#     row['Key_words'] = list(key_words_dict_scores.keys())

# # dropping the Plot column
# df.drop(columns = ['Plot'], inplace = True)

# df.head()

In [35]:
df.set_index('reference_id', inplace = True)   # set reference_id as the index of the dataframe
df.head()

,nft_id,name,asset_description,asset_contract_address,collection_name,collection_description,collection_slug,collection_verification,creator_username,creator_address,creator_profile_img_url,owner_username,owner_address,owner_profile_img_url,traits_string,total_rarity,num_sales,last_sale_timestamp,last_sale_total_price,last_sale_from_account_address,top_bid,open_sea_link,image_url
reference_id,,,,,,,,,,,,,,,,,,,,,,,
0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d-9974,9974,NaN,NaN,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,Bored Ape Yacht Club,The Bored Ape Yacht Club is a collection of 10...,boredapeyachtclub,True,BoredApeYachtClub,0xaba7161a7fb69c88e16ed9f455ce62b791ee4d03,https://storage.googleapis.com/opensea-static/...,mo31985,0x994cf01f34c51426bcb12bd30ff7079e280e1140,https://storage.googleapis.com/opensea-static/...,eyeseyepatch hatseaman'shat clothesrainbowsusp...,204.051791,2,2021-05-09T16:46:27,500000000000000000,0x994cf01f34c51426bcb12bd30ff7079e280e1140,NaN,https://opensea.io/assets/0xbc4ca0eda7647a8ab7...,https://lh3.googleusercontent.com/G60LFd7Xhb4-...
0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d-9973,9973,NaN,NaN,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,Bored Ape Yacht Club,The Bored Ape Yacht Club is a collection of 10...,boredapeyachtclub,True,BoredApeYachtClub,0xaba7161a7fb69c88e16ed9f455ce62b791ee4d03,https://storage.googleapis.com/opensea-static/...,accubed,0x6e0c6bf243232bcfef85a88eb61062626ffc1e2a,https://storage.googleapis.com/opensea-static/...,furdmt hatgirl'shairpink eyesrobot backgroundy...,182.516792,1,2021-05-01T16:24:22,580000000000000000,0x078ad2aa3b4527e4996d087906b2a3da51bba122,NaN,https://opensea.io/assets/0xbc4ca0eda7647a8ab7...,https://lh3.googleusercontent.com/-4Pz6B1FkH8k...
0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d-9949,9949,NaN,NaN,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,Bored Ape Yacht Club,The Bored Ape Yacht Club is a collection of 10...,boredapeyachtclub,True,BoredApeYachtClub,0xaba7161a7fb69c88e16ed9f455ce62b791ee4d03,https://storage.googleapis.com/opensea-static/...,NaN,0xd3b38f645ac56e0b8ab7b6071a1cfc12a6e8c05d,https://storage.googleapis.com/opensea-static/...,mouthgringoldgrill furtan hatseaman'shat eyes3...,221.356456,7,2021-11-21T10:04:02,85800000000000000000,0x91e1543bf18cc3c7a25e682f9c20cf8bd6f28548,NaN,https://opensea.io/assets/0xbc4ca0eda7647a8ab7...,https://lh3.googleusercontent.com/RcXAgekWgaH5...
0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d-9972,9972,NaN,NaN,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,Bored Ape Yacht Club,The Bored Ape Yacht Club is a collection of 10...,boredapeyachtclub,True,BoredApeYachtClub,0xaba7161a7fb69c88e16ed9f455ce62b791ee4d03,https://storage.googleapis.com/opensea-static/...,ARC001,0x0b31cc882a2b3937257ccf785b04b7c8f931207b,https://storage.googleapis.com/opensea-static/...,eyeshypnotized mouthrage backgroundaquamarine ...,149.528746,1,2021-05-01T11:03:09,150000000000000000,0x02eb75be1e72e988de64f0088d654d8ea1081e87,NaN,https://opensea.io/assets/0xbc4ca0eda7647a8ab7...,https://lh3.googleusercontent.com/vcL_TRKR8FRq...
0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d-9948,9948,NaN,NaN,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,Bored Ape Yacht Club,The Bored Ape Yacht Club is a collection of 10...,boredapeyachtclub,True,BoredApeYachtClub,0xaba7161a7fb69c88e16ed9f455ce62b791ee4d03,https://storage.googleapis.com/opensea-static/...,MAJE,0xecba5f51925e6ccec26da38dcd7d5305f6bdfbcb,https://storage.googleapis.com/opensea-static/...,hattrippycaptain'shat furcream eyesbloodshot m...,268.580759,2,2021-05-03T02:45:55,1750000000000000000,0xecba5f51925e6ccec26da38dcd7d5305f6bdfbcb,NaN,https://opensea.io/assets/0xbc4ca0eda7647a8ab7...,https://lh3.googleusercontent.com/jMsgIXhlR0Xe...


In [44]:
df.drop(columns = [col for col in df.columns if col!= 'traits_string'], inplace = True)
df.head()

,traits_string
reference_id,
0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d-9974,eyeseyepatch hatseaman'shat clothesrainbowsusp...
0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d-9973,furdmt hatgirl'shairpink eyesrobot backgroundy...
0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d-9949,mouthgringoldgrill furtan hatseaman'shat eyes3...
0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d-9972,eyeshypnotized mouthrage backgroundaquamarine ...
0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d-9948,hattrippycaptain'shat furcream eyesbloodshot m...


In [37]:
df['traits_string'].count()

10000

## Modeling

In order to detect similarities between movies, I need to vectorize, as I mentioned above. I decided to use CountVectorizer rather than TfIdfVectorizer for one simple reason: I need a simple frequency counter for each word in my bag_of_words column. Tf-Idf tends to give less importance to the words that are more present in the entire corpus (our whole column, in this case) which is not what we want for this application, because every word is important to detect similarity! 

*It seems like Tf-Idf would give more meaningful results* for ranking - check this later. For relevance, CountVectorizer seems to be ok.

Once I have the matrix containing the count for each word, we can apply the cosine_similarity function

In [38]:
# instantiating and generating the count matrix
count = CountVectorizer()   # used to transform a given text into a vector on the basis of the frequency (count) of each word that occurs in the entire text
count_matrix = count.fit_transform(df['traits_string'])

# print(count_matrix)
count_matrix.todense()

matrix([[1, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 1, 0, 0],
        [0, 1, 0, ..., 0, 0, 1],
        ...,
        [1, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [39]:
count_matrix.shape

(10000, 173)

in the count_matrix, i seems to be the movie, j seems to be each word in the bag_of_words per movie. the value at i, j gives the frequency of each word in the entire column of bag_of_words.

For some reason (need to find out) this frequency is needed to calculate the cosine similarity between the content in movies. - *The definition of similarity between two vectors u and v is, in fact, the ratio between their dot product and the product of their magnitudes.*

In [40]:
# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use later to match the indexes - easy to refer the movie title by index from the array that is created (cosine_sim)
indices = pd.Series(df.index)
indices[:5]

0    0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d-9974
1    0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d-9973
2    0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d-9949
3    0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d-9972
4    0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d-9948
Name: reference_id, dtype: object

Check if this concern can be addressed: Having cosine_sim in-memory can be a memory constraint for larger datasets

In [41]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
# cosine_sim.todense()

In [42]:
# function that takes in movie title as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    recommended_movies = []
    
    # getting the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])
        
    return recommended_movies

## Testing

In [43]:
recommendations('0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d-9948')

['0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d-7474',
 '0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d-633',
 '0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d-8279',
 '0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d-6352',
 '0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d-4343',
 '0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d-5900',
 '0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d-6706',
 '0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d-791',
 '0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d-5693',
 '0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d-6248']

# Custom Integration to rerank using Social Trends

Phrase-Based Matching

In [ ]:
trending_phrases = ['ROMCOM SERIES FOR HAESOO', 'Golden Corral', 'Wordle 228', 'Tietê', 'BOP TRACKLIST', '伊東純也', 'JIKJIN FOCUS FILM', 'NINJAPAN 2021', '一気読み', 'ムササビ', 'オーストラリア', 'WONPIL Concept Photo 3', 'Benzine 39', 'あーりん', 'البليهي', 'Kraft', '花枝ちゃん', 'HAPPY BIRTHDAY GEONU', 'ちゃん誕生日', 'サウジアラビア', '도영듀스', '凛ちゃん', 'SECRETCRUSH TRAILER', 'Best Entertainer Jungkook', 'UM CORPO DE VERÃO', 'Quito', 'Damn Brady', 'Schefter', '日村さん', 'SMILE FOR MEW', 'HBCUs', 'もやとらっしー']
# phrases contains all trending words

In [ ]:
import spacy
from spacy.matcher import PhraseMatcher

In [ ]:
nlp = spacy.load("en_core_web_sm")
matcher = PhraseMatcher(nlp.vocab)

# Only run nlp.make_doc to speed things up
patterns = [nlp.make_doc(text) for text in trending_phrases]
matcher.add("TerminologyList", patterns)
